In [5]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load datasets
customers = pd.read_csv("Customers.csv")
products = pd.read_csv("Products.csv")
transactions = pd.read_csv("Transactions.csv")

# Preprocessing Customers data
customers['SignupDate'] = pd.to_datetime(customers['SignupDate'])
customers['SignupYear'] = customers['SignupDate'].dt.year

# Aggregating Transactions data for Customer Profiles
customer_transactions = transactions.groupby('CustomerID').agg({
    'TotalValue': 'sum',
    'Quantity': 'sum',
    'ProductID': lambda x: ','.join(x),
}).reset_index()

# Merge customer data with aggregated transaction data
customer_profiles = pd.merge(customers, customer_transactions, on='CustomerID', how='left')

# Replace NaN values with empty strings for product data
customer_profiles['ProductID'] = customer_profiles['ProductID'].fillna('')

# Combine relevant features for similarity analysis
customer_profiles['ProfileData'] = (
    customer_profiles['Region'] + ' ' +
    customer_profiles['ProductID'] + ' ' +
    customer_profiles['SignupYear'].astype(str)
)

# Vectorizing the profile data using TF-IDF
vectorizer = TfidfVectorizer()
profile_matrix = vectorizer.fit_transform(customer_profiles['ProfileData'])

# Compute pairwise cosine similarity
similarity_matrix = cosine_similarity(profile_matrix)

# Generate recommendations for the first 20 customers
lookalike_results = {}

for idx, customer_id in enumerate(customer_profiles['CustomerID'][:20]):
    # Get similarity scores for this customer
    similarity_scores = list(enumerate(similarity_matrix[idx]))
    # Sort by similarity scores (excluding self-match)
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    similar_customers = [
        (customer_profiles.iloc[i]['CustomerID'], score)
        for i, score in similarity_scores[1:4]  # Top 3 matches
    ]
    # Store results in the lookalike map
    lookalike_results[customer_id] = similar_customers

# Save results to Lookalike.csv
lookalike_df = pd.DataFrame([
    {
        'CustomerID': cust_id,
        'Lookalikes': str([{'CustomerID': match[0], 'Score': match[1]} for match in matches])
    }
    for cust_id, matches in lookalike_results.items()
])

lookalike_df.to_csv('Vineet_Pundpal_Lookalike.csv', index=False)

# Display the lookalike results for validation
print(lookalike_df)


   CustomerID                                         Lookalikes
0       C0001  [{'CustomerID': 'C0190', 'Score': 0.4081232211...
1       C0002  [{'CustomerID': 'C0109', 'Score': 0.4005313760...
2       C0003  [{'CustomerID': 'C0181', 'Score': 0.4520943693...
3       C0004  [{'CustomerID': 'C0053', 'Score': 0.3393542823...
4       C0005  [{'CustomerID': 'C0096', 'Score': 0.4714393839...
5       C0006  [{'CustomerID': 'C0171', 'Score': 0.4875672231...
6       C0007  [{'CustomerID': 'C0140', 'Score': 0.4395196824...
7       C0008  [{'CustomerID': 'C0091', 'Score': 0.3550934824...
8       C0009  [{'CustomerID': 'C0062', 'Score': 0.4583669717...
9       C0010  [{'CustomerID': 'C0132', 'Score': 0.4135626855...
10      C0011  [{'CustomerID': 'C0177', 'Score': 0.3760730175...
11      C0012  [{'CustomerID': 'C0128', 'Score': 0.4189817298...
12      C0013  [{'CustomerID': 'C0058', 'Score': 0.3646297901...
13      C0014  [{'CustomerID': 'C0128', 'Score': 0.4678183264...
14      C0015  [{'Custome